In [7]:
from eins import EinsOp, Reductions as Red

import jax
import jax.numpy as jnp
import numpy as np
import torch

In [8]:
x =  torch.randn(1024, 256, 3)
y = torch.randn(1024, 256, 3)

z4 = EinsOp('b n1 d, b n2 d -> b n1 n2', combine='add', reduce=Red.l2_norm)(x, -y)

In [6]:
x: jax.Array = jnp.array(np.random.randn(1024, 256, 3))
y: jax.Array = jnp.array(np.random.randn(1024, 256, 3))

z4 = EinsOp('b n1 d, b n2 d -> b n1 n2', combine='add', reduce=Red.l2_norm)(x, -y)
# Version without eins. Note how easy it would be to write x[:, None, ...] - y[:, :, None, ...],
# which would lead to the transposed version of the pairwise distances you want.
z5 = jnp.sqrt(jnp.sum(jnp.square(x[:, :, None, ...] - y[:, None, ...]), axis=-1))

jnp.max(jnp.abs(z4 - z5))

Array(0., dtype=float32)

Signature: z4.view(dtype: 'DTypeLike | None' = None, type: 'None' = None) -> 'Array'
Docstring:
Return a bitwise copy of the array, viewed as a new dtype.

This is fuller-featured wrapper around :func:`jax.lax.bitcast_convert_type`.

If the source and target dtype have the same bitwidth, the result has the same
shape as the input array. If the bitwidth of the target dtype is different
from the source, the size of the last axis of the result is adjusted
accordingly.

>>> jnp.zeros([1,2,3], dtype=jnp.int16).view(jnp.int8).shape
(1, 2, 6)
>>> jnp.zeros([1,2,4], dtype=jnp.int8).view(jnp.int16).shape
(1, 2, 2)

Conversions involving booleans are not well-defined in all situations. With
regards to the shape of result as explained above, booleans are treated as
having a bitwidth of 8. However, when converting to a boolean array, the input
should only contain 0 or 1 bytes. Otherwise, results may be unpredictable or
may change depending on how the result is used.

This conversion is guaranteed 

In [3]:
@jax.jit
def jnp_pairwise_dist(x, y):
    return jnp.sqrt(jnp.sum(jnp.square(x[:, :, None, ...] - y[:, None, ...]), axis=-1))

@jax.jit
def ein_pairwise_dist(x, y):
    return EinsOp('b n1 d, b n2 d -> b n1 n2', combine='add', reduce=Red.l2_norm)(x, -y)

d1 = jnp_pairwise_dist(x, y)
d2 = ein_pairwise_dist(x, y)

In [4]:
%%timeit
jnp_pairwise_dist(x, y).block_until_ready()

63.5 ms ± 951 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
ein_pairwise_dist(x, y).block_until_ready()

69.7 ms ± 6.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
jax.make_jaxpr(jnp_pairwise_dist)(x, y)

{ lambda ; a:f32[1024,256,3] b:f32[1024,256,3]. let
    c:f32[1024,256,256] = pjit[
      jaxpr={ lambda ; d:f32[1024,256,3] e:f32[1024,256,3]. let
          f:f32[1024,256,1,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 1, 3)
            shape=(1024, 256, 1, 3)
          ] d
          g:f32[1024,1,256,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 2, 3)
            shape=(1024, 1, 256, 3)
          ] e
          h:f32[1024,256,256,3] = sub f g
          i:f32[1024,256,256,3] = integer_pow[y=2] h
          j:f32[1024,256,256] = reduce_sum[axes=(3,)] i
          k:f32[1024,256,256] = sqrt j
        in (k,) }
      name=jnp_pairwise_dist
    ] a b
  in (c,) }

In [7]:
jax.make_jaxpr(ein_pairwise_dist)(x, y)

{ lambda ; a:f32[1024,256,3] b:f32[1024,256,3]. let
    c:f32[1024,256,256] = pjit[
      jaxpr={ lambda ; d:f32[1024,256,3] e:f32[1024,256,3]. let
          f:f32[1024,256,3] = neg e
          g:f32[1024,256,1,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 1, 3)
            shape=(1024, 256, 1, 3)
          ] d
          h:f32[1024,1,256,3] = broadcast_in_dim[
            broadcast_dimensions=(0, 2, 3)
            shape=(1024, 1, 256, 3)
          ] f
          i:f32[1024,256,256,3] = add g h
          j:f32[1024,256,256,3] = integer_pow[y=2] i
          k:f32[1024,256,256] = reduce_sum[axes=(3,)] j
          l:f32[1024,256,256] = sqrt k
        in (l,) }
      name=ein_pairwise_dist
    ] a b
  in (c,) }

In [8]:
x.shape

(1024, 256, 3)

In [9]:
op = EinsOp('a b c -> a b c 4')

In [10]:
op(x)

ValueError: ['a', 'b', 'c'] != ['a', 'b', 'c', '4']

In [ ]:
type(op.program.constr.free_vars[0])

str